In [20]:
import pandas as pd
import pprint as pp

url_jobs = "https://github.com/MatthewjRay/DP_StudyPlan/raw/main/data_cleaned_2021%202.csv"
df1 = pd.read_csv(url_jobs,index_col=0)
job_data = df1
url_course = "https://raw.githubusercontent.com/MatthewjRay/DP_StudyPlan/main/Courses_Levels_Hours.csv"  # Replace with the actual URL
df2 = pd.read_csv(url_course, sep=',')
course_data = df2

available_roles = [
    'Data Scientist',
    'Data Analitics',
    'Data Engineer',
    'Data Modeler',
    'Analyst',
    'Machine Learning Engineer',
    'Project Manager']

print("Please choose a role from the following list:")
for role in available_roles:
    print(role)

chosen_role = input("Enter the role you are interested in: ")
filtered_jobs = job_data[job_data['job_title_sim'].str.contains(chosen_role, case=False)]
skills = ['Python', 
          'spark', 
          'aws', 
          'excel', 
          'sql', 
          'sas', 
          'keras', 
          'pytorch', 
          'scikit', 
          'tensor', 
          'hadoop', 
          'tableau', 
          'bi', 
          'flink', 
          'mongo', 
          'google_an']

study_hours = int(input("Enter the number of preparation hours: "))

skill_importance = {}
for skill in skills:
    skill_importance[skill] = filtered_jobs[skill].sum()

total_importance = sum(skill_importance.values())
importance_values = {skill: ((importance / total_importance) * 100 if total_importance != 0 else 0) for skill, importance in skill_importance.items()}

sorted_skills = sorted(skill_importance.items(), key=lambda x: x[1], reverse=True)
skill_levels = len(sorted_skills)
level_1_threshold = int(skill_levels * 0.6)
level_2_threshold = int(skill_levels * 0.3)

competency_levels = {}
for index, (skill, importance) in enumerate(sorted_skills):
    if index < level_2_threshold:
        competency_levels[skill] = 3
    elif index < level_1_threshold:
        competency_levels[skill] = 2
    else:
        competency_levels[skill] = 1

courses = []
for index, row in course_data.iterrows():
    course = {
        "subject": row["Course"],
        "level": row["Level"],
        "hours": row["Hours"],
        "value": importance_values[row["Course"]] * competency_levels[row["Course"]]  # Multiply importance value by competency level
    }
    courses.append(course)

relevant_courses = []
for skill, level in competency_levels.items():
    for course in courses:
        if course["subject"] == skill and course["level"] == level:
            relevant_courses.append(course)
            break

relevant_courses = sorted(relevant_courses, key=lambda x: x["value"], reverse=True)

class StudyPlanBasic:
    def dp_algo(self, course_list, time_left_for_interview, index=0, memo=None):
        if memo is None:
            memo = {}
        if index == len(course_list):
            return [], 0
        if (time_left_for_interview, index) in memo:
            return memo[(time_left_for_interview, index)]

        courses_taken_skip, value_acquired_skip = self.dp_algo(course_list, time_left_for_interview, index + 1, memo)

        course = course_list[index]
        if course["hours"] <= time_left_for_interview:
            courses_taken_take, value_acquired_take = self.dp_algo(
                course_list, time_left_for_interview - course["hours"], index + 1, memo
            )
            value_acquired_take += course["value"]
            courses_taken_take.append(
                {
                    "course": course["subject"],
                    "level": course["level"],
                    "percentage_value_acquired": round((value_acquired_take / course["value"]) * 100, 2) if course["value"] != 0 else 0,
                    "time_spend": course["hours"]
                })

            memo[(time_left_for_interview, index)] = courses_taken_take, value_acquired_take
        else:
            fraction = time_left_for_interview / course["hours"]
            value_acquired_fraction = course["value"] * fraction
            courses_taken_fraction = [
                {
                    "course": course["subject"],
                    "level": course["level"],
                    "percentage_value_acquired": round((value_acquired_fraction / course["value"]) * 100, 2) if course["value"] != 0 else 0,
                    "time_spend": time_left_for_interview
                }]
            memo[(time_left_for_interview, index)] = courses_taken_fraction, value_acquired_fraction

        if memo[(time_left_for_interview, index)][1] < value_acquired_skip:
            memo[(time_left_for_interview, index)] = courses_taken_skip, value_acquired_skip

        return memo[(time_left_for_interview, index)]
if __name__ == "__main__":
    plan = StudyPlanBasic()
    total_hrs = study_hours
    study_plan, value_acquired = plan.dp_algo(relevant_courses, total_hrs)
    max_possible_value = sum(course["value"] for course in relevant_courses)
    pcnt_total_value_acquired = round((value_acquired / max_possible_value) * 100, 2) if max_possible_value != 0 else 0

    print("Plan for the interview looks like below(time in hrs):")
    study_plan = [course for course in study_plan if course["time_spend"] > 0]
    study_plan = sorted(study_plan, key=lambda x: (x["percentage_value_acquired"], x["level"]), reverse=True)
    pp.pprint(study_plan)
    print(f"Percentage chance to crack the interview with {total_hrs} hours with the plan is: {pcnt_total_value_acquired}%")

Please choose a role from the following list:
Data Scientist
Data Analitics
Data Engineer
Data Modeler
Analyst
Machine Learning Engineer
Project Manager
Enter the role you are interested in: Project Manager
Enter the number of preparation hours: 250
Plan for the interview looks like below(time in hrs):
[{'course': 'tableau',
  'level': 3,
  'percentage_value_acquired': 318.89,
  'time_spend': 60},
 {'course': 'excel',
  'level': 3,
  'percentage_value_acquired': 238.79,
  'time_spend': 60},
 {'course': 'sql',
  'level': 3,
  'percentage_value_acquired': 138.79,
  'time_spend': 60},
 {'course': 'Python',
  'level': 3,
  'percentage_value_acquired': 106.67,
  'time_spend': 60},
 {'course': 'aws',
  'level': 2,
  'percentage_value_acquired': 20.0,
  'time_spend': 10}]
Percentage chance to crack the interview with 250 hours with the plan is: 97.29%
